In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mne
from mne.io import read_raw_ant
from mne.utils.docs import groups
import antio
import importlib
from typing import List
from pathlib import Path

import combined_analysis_bachelor.code.functions_for_pipeline as funcs
import combined_analysis_bachelor.code.movement_detection_functions as move_functions
from utils.get_sub_dir import get_sub_folder_dir
from combined_analysis_bachelor.code.create_processed_files import create_processed_files

## starting with splitting continous recordings ##

In [2]:
#filepath_E_source = "E:/classification_cnt/sub_class_E.cnt"
#E = read_raw_ant(filepath_E_source, preload=True)
#onsets_E = E.annotations.onset
#label_dict_E = {
#    "rest1": [onsets_E[0], onsets_E[1]],
#    "rest2": [onsets_E[3], onsets_E[4]],
#    "rest3": [onsets_E[6], onsets_E[7]],
#    "move1": [onsets_E[9], onsets_E[10]],
#    "move2": [onsets_E[12], onsets_E[13]],
#    "move3": [onsets_E[15], onsets_E[16]],
#    "suppr1": [onsets_E[-8], onsets_E[-7]],
#    "suppr2": [onsets_E[-5], onsets_E[-4]],
#    "suppr3": [onsets_E[-2], onsets_E[-1]],
#}
#
#filepath_F_source = "E:/classification_cnt/sub_class_F.cnt"
#F = read_raw_ant(filepath_F_source, preload=True)
#onsets_F = F.annotations.onset
#label_seq_F = ["rest", "rest", "rest", "move", "move", "move", "fail", "suppr", "suppr", "suppr"]
#label_dict_F = {
#    "rest1": [onsets_F[0], onsets_F[1]],
#    "rest2": [onsets_F[3], onsets_F[4]],
#    "rest3": [onsets_F[6], onsets_F[7]],
#    "move1": [onsets_F[9], onsets_F[10]],
#    "move2": [onsets_F[12], onsets_F[13]],
#    "move3": [onsets_F[15], onsets_F[16]],
#    "suppr1": [onsets_F[-8], onsets_F[-7]],
#    "suppr2": [onsets_F[-5], onsets_F[-4]],
#    "suppr3": [onsets_F[-2], onsets_F[-1]],
#}
#
#filepath_L_source = "E:/classification_cnt/sub_class_L.cnt"
#L = read_raw_ant(filepath_L_source, preload=True)
#onsets_L = L.annotations.onset
#label_seq_L = ["rest", "move", "suppr", "rest", "move", "suppr", "rest", "move", "suppr"]
#label_dict_L = {
#    "rest1": [onsets_L[0], onsets_L[1]],
#    "move1": [onsets_L[3], onsets_L[4]],
#    "suppr1": [onsets_L[6], onsets_L[7]],
#    "rest2": [onsets_L[9], onsets_L[10]],
#    "move2": [onsets_L[12], onsets_L[13]],
#    "suppr2": [onsets_L[15], onsets_L[16]],
#    "rest3": [onsets_L[-8], onsets_L[-7]],
#    "move3": [onsets_L[-5], onsets_L[-4]],
#    "suppr3": [onsets_L[-2], onsets_L[-1]],
#}
#
#filepath_P_source = "E:/classification_cnt/sub_class_P.cnt"
#P = read_raw_ant(filepath_P_source, preload=True)
#onsets_P = P.annotations.onset
#label_seq_P = ["rest", "rest", "rest", "move", "move", "move", "suppr", "suppr", "suppr"]
#label_dict_P = {
#    "rest1": [onsets_P[0], onsets_P[1]],
#    "rest2": [onsets_P[3], onsets_P[4]],
#    "rest3": [onsets_P[6], onsets_P[7]],
#    "move1": [onsets_P[9], onsets_P[10]],
#    "move2": [onsets_P[12], onsets_P[13]],
#    "move3": [onsets_P[15], onsets_P[16]],
#    "suppr1": [onsets_P[-8], onsets_P[-7]],
#    "suppr2": [onsets_P[-5], onsets_P[-4]],
#    "suppr3": [onsets_P[-2], onsets_P[-1]],
#}
#
#classification_data_path = get_sub_folder_dir(sub="classification", data_subfolder="classification_data")
#target_dir = Path(f"{classification_data_path}/class_raw")
#
#channels_class = ["BIP3", "BIP4", "BIP5", "BIP7", "BIP8", "BIP9", "BIP10"]
#EMG = ["BIP7", "BIP8", "BIP9", "BIP10"]
#EMG_names = ["brachioradialis_L", "deltoideus_L"]
#ACC = ["acc_x_hand_L", "acc_y_hand_L", "acc_z_hand_L"]
#LOCS_left = {"BIP7":"brachioradialis_L",
#            "BIP8": "deltoideus_L",
#            "BIP3" : "acc_x_hand_L",
#            "BIP4" : "acc_y_hand_L",
#            "BIP5" : "acc_z_hand_L"}
#LOCS_right = {"BIP3" : "acc_x_hand_L",
#            "BIP4" : "acc_y_hand_L",
#            "BIP5" : "acc_z_hand_L",
#            "BIP9" : "brachioradialis_R",
#            "BIP10" : "deltoideus_R"}

In [4]:
#move_functions.split_recording_by_manual_segments(
#    filepath=filepath_P_source,
#    output_dir=target_dir,
#    segments_dict=label_dict_P,
#    channels= channels_class,
#    location_dict=LOCS_left,
#)

### we have just splitted the .cnt files and converted them into h5 ###
### Now let's process the files (notch and bandpass filter)! ###

In [2]:
classification_data_path = get_sub_folder_dir(sub="classification", data_subfolder="classification_data")
source_dir = Path(f"{classification_data_path}/class_raw")
target_processed_dir = Path(f"{classification_data_path}/class_processed")
EMG_names = ["brachioradialis_L", "deltoideus_L"]

['Charité - Universitätsmedizin Berlin', 'OneDrive - Charité - Universitätsmedizin Berlin']


In [50]:
#create_processed_files(source_dir, target_processed_dir, [1,20], [20,450])

#### and lets look at the data ####

In [3]:
#comparing forearm EMG values for two subs # 
check_P_rest_proc = f"{target_processed_dir}/sub_class_P_rest1_processed.h5"
check_rest_proc_P_df = pd.read_hdf(check_P_rest_proc, key="data")

check_F_rest_proc = f"{target_processed_dir}/sub_class_F_rest1_processed.h5"
check_rest_proc_F_df = pd.read_hdf(check_F_rest_proc, key="data")

#looking at movement for sub_P
check_move = f"{target_processed_dir}/sub_class_P_move2_processed.h5"
check_move_df = pd.read_hdf(check_move, key="data")

In [4]:
check_rest_proc_P_df

,brachioradialis_L,deltoideus_L,time_sec,SVM
0,2.984279e-12,-5.060397e-13,0.000,-1.561251e-17
1,6.753690e+01,-6.283881e+00,0.001,1.864461e-03
2,1.175139e+02,1.004827e+01,0.002,3.725657e-03
3,1.089274e+02,5.754423e+00,0.003,5.576344e-03
4,2.340514e+01,6.290831e+00,0.004,7.411539e-03
...,...,...,...,...
30202,2.861575e+01,-5.981973e+01,30.202,2.946091e-02
30203,1.149216e+01,-4.690266e+01,30.203,2.230545e-02
30204,-1.898056e+01,-3.779000e+01,30.204,1.497104e-02
30205,-2.921936e+01,-1.053778e+01,30.205,7.516269e-03


In [6]:
# comparing filtered data from different subs
fig, axs = plt.subplots(2, 1)

#axs[0].plot(check_rest_proc_P_df["time_sec"], check_rest_proc_P_df["brachioradialis_L"])
axs[0].set_title("sub_P rest (brachioradialis)")
axs[0].set_ylabel("Amplitude (microV)")

#axs[1].plot(check_rest_proc_F_df["time_sec"], check_rest_proc_F_df["brachioradialis_R"])
axs[1].set_title("sub_F rest (brachioradialis)")
axs[1].set_ylabel("Amplitude (microV)")

plt.tight_layout()
plt.show()

In [7]:
# data has bigger vals at start and end --> check out how data looks with cropped edges
cropped_rest_P = funcs.crop_edges(check_rest_proc_P_df)
cropped_rest_F = funcs.crop_edges(check_rest_proc_F_df)
check_move_P = funcs.crop_edges(check_move_df)

In [53]:
# plotting cropped data
fig, axs = plt.subplots(2, 1)

axs[0].plot(cropped_rest_P["time_sec"], cropped_rest_P["brachioradialis_L"])
axs[0].set_title("sub_P rest (brachioradialis)")
axs[0].set_ylabel("Amplitude (microV)")

axs[1].plot(cropped_rest_F["time_sec"], cropped_rest_F["brachioradialis_R"])
axs[1].set_title("sub_F rest (brachioradialis)")
axs[1].set_ylabel("Amplitude (microV)")

plt.tight_layout()
plt.show()

In [9]:
# testing new function: adding smoothed ACC to dataset # 
ACC_col_add = ["SVM"]
df_added = funcs.apply_savgol_rms_acc(check_move_P, ACC_col_add, savgol_window_length=21, savgol_polyorder=3, rms_window_size=100)

In [6]:
# df_added

In [11]:
# plotting different SVM cols to see if function worked # 
plt.figure()
plt.plot(df_added["time_sec"], df_added["SVM"], label="SVM")
plt.plot(df_added["time_sec"], df_added["SVM_smooth_rms"], label="SVM smoothed rms")
plt.legend()
plt.show() 

# creating one big training df #

In [10]:
# getting all of the filepaths from the folder:
from combined_analysis_bachelor.code.read_in_emg_acc import get_filepaths_filenames
from utils.find_paths import get_onedrive_path
SUB = "classification"

directory = get_onedrive_path(folder="classification_data")
class_filepaths, _ = get_filepaths_filenames(SUB, directory)
print(directory)

['Charité - Universitätsmedizin Berlin', 'OneDrive - Charité - Universitätsmedizin Berlin']
C:\Users\User\OneDrive - Charité - Universitätsmedizin Berlin\TRR295 via Teams - LID_MEG - LID_MEG\data\classification_data


In [11]:
# class_filepaths

In [12]:
df_dir = f"{directory}/sub-classification/EMG_ACC/training_dfs/training_df_full.h5"

In [13]:
window_size = 250
overlap = 0.5
SF = 1000

# move_functions.create_train_df(class_filepaths, window_size, overlap, SF, df_dir)

In [14]:
df_train = pd.read_hdf(df_dir, key="data")

In [27]:
df_train

,sub_ID,label,window_range_ms,brachioradialis_rms,brachioradialis_mean,brachioradialis_std,brachioradialis_max,brachioradialis_zeroCrossings,brachioradialis_slopeSignChanges,brachioradialis_waveformLength,...,SVM_smooth_rms_mean,SVM_smooth_rms_std,SVM_smooth_rms_max,SVM_smooth_rms_min,SVM_smooth_rms_range,SVM_smooth_rms_coefVar,SVM_smooth_rms_psd_peakfreq,SVM_smooth_rms_psd_power_1_5Hz,SVM_smooth_rms_psd_power_10_20Hz,SVM_smooth_rms_psd_power_ratio_high_low
0,sub-E,move,0-250ms,72.636121,0.982523,72.629475,186.236991,36,68,5398.107678,...,0.121205,0.076269,0.234294,0.032451,0.201843,0.629253,4.0,5.604309e-04,4.593030e-06,0.008181
1,sub-E,move,125-375ms,61.170302,-1.159515,61.159311,187.257614,38,75,4997.888572,...,0.181053,0.044604,0.234294,0.089483,0.144811,0.246361,4.0,2.524838e-04,1.945152e-07,0.000767
2,sub-E,move,250-500ms,103.284739,1.085663,103.279033,227.489370,27,57,7481.488226,...,0.153216,0.041979,0.232519,0.090045,0.142474,0.273987,4.0,2.744235e-04,8.759492e-06,0.031804
3,sub-E,move,375-625ms,93.819652,2.468111,93.787182,227.489370,31,51,6645.862774,...,0.153420,0.032716,0.221536,0.090045,0.131491,0.213245,4.0,6.655335e-05,2.690929e-06,0.039834
4,sub-E,move,500-750ms,53.071385,-1.162224,53.058658,129.990476,33,55,4720.779205,...,0.198114,0.040617,0.265608,0.122671,0.142937,0.205016,4.0,2.984760e-04,1.506785e-06,0.005031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,sub-P,suppr,27875-28125ms,121.118164,-0.496168,121.117148,340.138893,40,97,11606.611817,...,0.005544,0.001458,0.008165,0.003418,0.004746,0.263004,4.0,1.610816e-07,6.647380e-08,0.057252
224,sub-P,suppr,28000-28250ms,139.382221,1.977951,139.368186,466.308132,39,90,14887.293215,...,0.003686,0.000928,0.005292,0.001499,0.003793,0.251900,8.0,5.346998e-09,4.965878e-09,0.004939
225,sub-P,suppr,28125-28375ms,131.158581,1.823825,131.145899,466.308132,39,88,13594.655779,...,0.003927,0.001146,0.005387,0.001499,0.003888,0.291776,4.0,2.449542e-07,5.215440e-08,0.041893
226,sub-P,suppr,28250-28500ms,114.961409,-7.096147,114.742191,392.345539,45,89,11595.355079,...,0.004274,0.000986,0.005387,0.002398,0.002989,0.230674,4.0,1.537980e-07,3.138563e-08,0.027202


In [15]:
feature_cols = [i for i in df_train.columns if i not in ["label", "sub_ID", "window_range_ms"]]

In [16]:
len(feature_cols)

48

#### next step: splitting train_df according to Leave-One-Subject-Out and scaling the data ####

In [17]:
importlib.reload(move_functions)

<module 'combined_analysis_bachelor.code.movement_detection_functions' from 'C:\\Users\\User\\Documents\\GitHub\\lid_opm\\combined_analysis_bachelor\\code\\movement_detection_functions.py'>

In [18]:
loso_data_old = move_functions.prepare_loso_data(df_train)

In [19]:
# check if that worked
for fold in loso_data_old:
    print("Testsubject:", fold["test_sub_ID"])
    print("Trainshape:", fold["X_train"].shape)
    print("Testshape:", fold["X_test"].shape)

Testsubject: sub-E
Trainshape: (6246, 48)
Testshape: (2039, 48)
Testsubject: sub-F
Trainshape: (6166, 48)
Testshape: (2119, 48)
Testsubject: sub-L
Trainshape: (6224, 48)
Testshape: (2061, 48)
Testsubject: sub-P
Trainshape: (6219, 48)
Testshape: (2066, 48)


#### feature selection by feature permutation ####

In [112]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# === PARAMETERS ===
IMPORTANCE_THRESHOLD = 0.01
HIGH_ACCURACY_THRESHOLD = 0.95

# === INITIALIZE ===
all_feature_scores = pd.Series(0, index=feature_cols)
feature_cols = [i for i in df_train.columns if i not in ["label", "sub_ID", "window_range_ms"]]
fold_accuracies = []
fold_feature_lists = []

# === loop through each fold ===
for fold_idx, fold in enumerate(loso_data_old):
    print(f"\n--- Fold {fold_idx+1}/{len(loso_data_old)} ---")
    
    X_train = pd.DataFrame(fold['X_train'], columns=feature_cols)
    X_val = pd.DataFrame(fold['X_test'], columns=feature_cols)
    y_train = fold['y_train']
    y_val = fold['y_test']

    # training the model
    model = KNeighborsClassifier()
    model.fit(X_train, y_train)

    # Accuracy
    acc = accuracy_score(y_val, model.predict(X_val))
    fold_accuracies.append(acc)
    print(f"Accuracy: {acc:.3f}")

    # Permutation Importance
    importances = move_functions.permutation_importance(model, X_val, y_val)
    important_feature_indices = importances[importances > IMPORTANCE_THRESHOLD].index.tolist()
    fold_feature_lists.append(important_feature_indices)

    print(f"important features in fold {fold_idx+1}: {important_feature_indices}")

# === feature selection strategy ===
final_selected_indices = set()
for acc, features in zip(fold_accuracies, fold_feature_lists):
    if acc >= HIGH_ACCURACY_THRESHOLD or len(features) > 0:
        final_selected_indices.update(features)

# === index to feat name ===
final_selected_features = list(final_selected_indices)

print("\n=====> Final Features over all Folds (combined):")
print(final_selected_features)


--- Fold 1/4 ---
Accuracy: 0.974
important features in fold 1: ['brachioradialis_slopeSignChanges', 'brachioradialis_envelope_coefVar', 'SVM_smooth_rms_min']

--- Fold 2/4 ---
Accuracy: 0.988
important features in fold 2: ['brachioradialis_slopeSignChanges', 'brachioradialis_waveformLength', 'deltoideus_rms', 'deltoideus_std', 'deltoideus_max', 'deltoideus_waveformLength', 'brachioradialis_envelope_coefVar', 'deltoideus_tkeo_rms', 'deltoideus_tkeo_mean', 'deltoideus_tkeo_std', 'deltoideus_tkeo_max', 'deltoideus_tkeo_waveformLength', 'deltoideus_envelope_rms', 'deltoideus_envelope_mean', 'deltoideus_envelope_std', 'deltoideus_envelope_max']

--- Fold 3/4 ---
Accuracy: 0.988
important features in fold 3: ['brachioradialis_rms', 'brachioradialis_mean', 'brachioradialis_std', 'brachioradialis_max', 'brachioradialis_slopeSignChanges', 'brachioradialis_envelope_rms', 'brachioradialis_envelope_mean', 'brachioradialis_envelope_std', 'brachioradialis_envelope_max']

--- Fold 4/4 ---
Accuracy: 

In [20]:
final_features = ['SVM_smooth_rms_max', 'SVM_smooth_rms_mean', 'SVM_smooth_rms_min', 'SVM_smooth_rms_psd_power_ratio_high_low', 'SVM_smooth_rms_range', 'SVM_smooth_rms_std', 'brachioradialis_envelope_coefVar', 'brachioradialis_envelope_max', 'brachioradialis_envelope_mean', 'brachioradialis_envelope_rms', 'brachioradialis_max', 'brachioradialis_rms', 'brachioradialis_slopeSignChanges', 'brachioradialis_std', 'brachioradialis_tkeo_max', 'brachioradialis_tkeo_mean', 'brachioradialis_tkeo_rms', 'brachioradialis_tkeo_slopeSignChanges', 'brachioradialis_tkeo_std', 'brachioradialis_tkeo_waveformLength', 'brachioradialis_waveformLength', 'deltoideus_envelope_coefVar', 'deltoideus_envelope_max', 'deltoideus_max', 'deltoideus_rms', 'deltoideus_slopeSignChanges', 'deltoideus_std', 'deltoideus_tkeo_mean', 'deltoideus_tkeo_rms', 'deltoideus_tkeo_waveformLength', 'deltoideus_waveformLength']

drop_feats = [i for i in df_train.columns if i not in final_features and i not in ["label", "sub_ID", "window_range_ms"]]

print(len(df_train.columns))
print(len(final_features))
print(len(drop_feats))

51
31
17


In [21]:
df_picked_feats_dir = f"{directory}/sub-classification/EMG_ACC/training_dfs/training_df_final_feats.h5"

df_train_final_feats = df_train.drop(drop_feats, axis=1)
df_train_final_feats.to_hdf(df_picked_feats_dir, key="data")

In [22]:
loso_data = move_functions.prepare_loso_data(df_train_final_feats)

### 1. Model with cross_val: logistic regression ###

In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [105]:
move_functions.fit_model_acc_sens_spec(loso_data, model_name="logistic_regression")

logistic_regression Accuracy Ø: 0.984
Accuracy for each fold: [0.986 0.976 0.984 0.991]
rest: sensitivity Ø = 0.972, specificity Ø = 0.994
move: sensitivity Ø = 0.992, specificity Ø = 0.996
suppr: sensitivity Ø = 0.988, specificity Ø = 0.987


In [48]:
move_functions.plot_loso_roc(loso_data, model_name="logistic_regression", class_names=["rest", "move", "suppr"], mode="one-vs-all")

In [29]:
importlib.reload(move_functions)

<module 'combined_analysis_bachelor.code.movement_detection_functions' from 'C:\\Users\\User\\Documents\\GitHub\\lid_opm\\combined_analysis_bachelor\\code\\movement_detection_functions.py'>

In [30]:
move_functions.plot_cm(model_name="logistic_regression", folds_data=loso_data, sub_id="sub-E")

### 2. Model with cross_val : KNN ###

In [106]:
move_functions.fit_model_acc_sens_spec(loso_data, model_name="KNN")

KNN Accuracy Ø: 0.969
Accuracy for each fold: [0.983 0.988 0.991 0.913]
rest: sensitivity Ø = 0.980, specificity Ø = 0.992
move: sensitivity Ø = 0.936, specificity Ø = 0.996
suppr: sensitivity Ø = 0.990, specificity Ø = 0.965


In [ ]:
# plot: KNN aufteilung, für sub mit bester klassifizierung? 

In [31]:
move_functions.plot_cm(model_name="KNN", folds_data=loso_data, sub_id="sub-L")

In [73]:
# get plots where different k-vals are used:
fold_P = loso_data[2]
X_test_P = fold_P["X_test"]
X_train_P = fold_P["X_train"]
y_test_P = fold_P["y_test"]
y_train_P = fold_P["y_train"]

move_functions.plot_knn_predictions_clean(X_train_P, X_test_P, y_train_P, y_test_P, test_sub="sub-P", k=5) 

C:\Users\User\Documents\GitHub\lid_opm\combined_analysis_bachelor\code\movement_detection_functions.py:282: UserWarning: You passed a edgecolor/edgecolors ('black') for an unfilled marker ('x').  Matplotlib is ignoring the edgecolor in favor of the facecolor.  This behavior may change in the future.
  plt.scatter(X_test_pca[i, 0], X_test_pca[i, 1],


0.4393530997304582

### 3. Model with cross_val: Random Forest ###

In [123]:
move_functions.fit_model_acc_sens_spec(loso_data, model_name="random_forest")

random_forest Accuracy Ø: 0.918
Accuracy for each fold: [0.996 0.998 0.981 0.698]
rest: sensitivity Ø = 0.989, specificity Ø = 0.992
move: sensitivity Ø = 0.778, specificity Ø = 0.999
suppr: sensitivity Ø = 0.983, specificity Ø = 0.885


In [16]:
# spannend, wenn Patrick als test_sub genommen wurde, ist die accuracy deutlich schlechter, würde man ihn raus nehmen, wäre die mean accuracy bestimmt am besten von allen models:

In [ ]:
# das heißt aber auch: da wo er im training mit dabei war, ist die performance trotzdem richtig gut. Macht ja aber denke ich sinn weil: seine daten (besonders forearm) waren am noisiesten --> wenn also modell nur mit daten trainert wird, die wenig noisy sind, und test sub aber eher noisy daten hat, dass wahrscheinlich rest vom test_sub oft falsch als movement eingestuft wird würde ich mal vermuten? Kann ich das mit confusion matrix zeigen?: 

In [32]:
move_functions.plot_cm(model_name="random_forest", folds_data=loso_data, sub_id="sub-P")

### jetzt einmal mit PCA die einordnungen der subs sehen ###

In [56]:
importlib.reload(move_functions)

fold = [f for f in loso_data if f["test_sub_ID"] == "sub-P"][0]
feature_cols=[col for col in df_train.columns if col not in ["sub_ID", "label", "window_range_ms"]]

pca = move_functions.plot_subject_feature_space(df_train, fold, feature_cols=[col for col in df_train.columns if col not in ["sub_ID", "label", "window_range_ms"]])

In [44]:
loadings = pca.components_.T * np.sqrt(pca.explained_variance_)
print("Loadings:")
print(loadings)

plt.figure(figsize=(10, 8))
sns.heatmap(loadings, annot=True, cmap='coolwarm', xticklabels=['PC1', 'PC2'], yticklabels=feature_cols)
plt.title('Feature Importance in Principal Components')
plt.tight_layout()
plt.show()

Loadings:
[[ 4.17798765e-01  8.39130642e-01]
 [-6.43531320e-03  2.79253889e-03]
 [ 4.17801027e-01  8.39150027e-01]
 [ 3.64733670e-01  8.98561767e-01]
 [-6.58472782e-01 -1.49928530e-01]
 [-5.34336329e-01 -3.03401995e-01]
 [ 2.94940074e-01  8.45376334e-01]
 [ 6.93551405e-01 -1.70212128e-01]
 [ 5.70715211e-03 -1.17076393e-02]
 [ 6.93518633e-01 -1.70222218e-01]
 [ 6.73056189e-01 -1.68303290e-01]
 [-6.28469637e-01  7.63616559e-02]
 [-7.41536692e-01  1.96258987e-02]
 [ 6.35149821e-01 -1.89682042e-01]
 [ 2.84471380e-01  9.94133896e-01]
 [ 2.75341549e-01  9.27774238e-01]
 [ 2.87212718e-01  1.06400819e+00]
 [ 2.89162081e-01  1.04669402e+00]
 [ 9.02873942e-38  0.00000000e+00]
 [-8.30902986e-02 -5.44403140e-01]
 [ 2.58847632e-01  9.99343208e-01]
 [ 4.37925903e-01  7.98098537e-01]
 [ 4.29858734e-01  8.02314213e-01]
 [ 5.23392225e-01  3.50062023e-01]
 [ 4.87442573e-01  7.55557906e-01]
 [ 5.18601505e-01 -8.86120867e-02]
 [ 4.33386869e-01 -1.69898102e-01]
 [ 4.39027275e-01 -1.71433335e-01]
 [ 4.18613

In [103]:
importlib.reload(move_functions)

<module 'combined_analysis_bachelor.code.movement_detection_functions' from 'C:\\Users\\User\\Documents\\GitHub\\lid_opm\\combined_analysis_bachelor\\code\\movement_detection_functions.py'>

In [64]:
# Split-Data (from one fold)
X_train = fold["X_train"]
y_train = fold["y_train"]
X_test = fold["X_test"]
y_test = fold["y_test"]
test_sub = fold["test_sub_ID"]
groups = fold["groups"]

X_scaled = np.vstack([X_train, X_test])
y = np.hstack([y_train, y_test])


# PCA-Plot with svm
move_functions.plot_pca_with_svm(X_scaled, y, groups, test_sub="sub-P", labels=["move", "rest", "suppr"])

In [61]:
importlib.reload(move_functions)

fold = [f for f in loso_data if f["test_sub_ID"] == "sub-F"][0]
move_functions.plot_subject_feature_space(df_train, fold, feature_cols=[col for col in df_train.columns if col not in ["sub_ID", "label", "window_range_ms"]])

,n_components,2
,copy,True
,whiten,False
,svd_solver,'auto'
,tol,0.0
,iterated_power,'auto'
,n_oversamples,10
,power_iteration_normalizer,'auto'
,random_state,None


#### PCA for all folds ####

In [62]:
move_functions.plot_loso_pca(loso_data, feature_cols, n_top_features=20)


Top 20 positive Features for PC1:
                         feature      PC1      PC2
                  deltoideus_rms 0.234221 0.092280
                  deltoideus_std 0.234218 0.092284
         deltoideus_envelope_max 0.234116 0.094629
         deltoideus_envelope_rms 0.233357 0.091184
        deltoideus_envelope_mean 0.232675 0.090254
                  deltoideus_max 0.230463 0.091956
       deltoideus_waveformLength 0.224243 0.100340
            deltoideus_tkeo_mean 0.194305 0.102658
     deltoideus_slopeSignChanges 0.194152 0.032853
             deltoideus_tkeo_rms 0.193250 0.101824
         deltoideus_envelope_std 0.192872 0.092462
  deltoideus_tkeo_waveformLength 0.192149 0.101577
             deltoideus_tkeo_max 0.188617 0.098700
             deltoideus_tkeo_std 0.187828 0.098258
deltoideus_tkeo_slopeSignChanges 0.145527 0.045997
brachioradialis_slopeSignChanges 0.144516 0.103121
             SVM_smooth_rms_mean 0.143125 0.067963
              SVM_smooth_rms_max 0.140269 0.070

## processing external val data for final tests ##